#### Imports

In [417]:
import pandas as pd
import numpy as np
import re

#### Read linkedin csv

In [418]:
linkedin = pd.read_csv('../data/linkedin.csv')
linkedin.sample(3)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
51,3752496774,AI/ML engineer,First Soft Solutions LLC,"· Dallas, TX",78 $/h - 104 $/h,Híbrido,Contrato por obra,Intermedio,De 51 a 200 empleados · Servicios y consultorí...
281,3775623213,Junior Data Analyst,Coders Data,"· Texas, Estados Unidos",20 $/h - 24 $/h,En remoto,Contrato por obra,Sin experiencia,De 201 a 500 empleados · Servicios y consultor...
288,3775816825,Artificial Intelligence Engineer,Braintrust,· Estados Unidos,15 $/h - 35 $/h,En remoto,Jornada completa,Sin experiencia,"De 11 a 50 empleados · Tecnología, información..."


#### Cleaning 'company_state' column

In [419]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'NC', 'SC', 'CO', 'CT', 'ND', 'SD', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
          'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NJ', 'NY', 'NH', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'TN', 'TX', 'UT',
          'UT', 'VT', 'VA', 'WV', 'WA', 'WI', 'WY']
linkedin['company_state'] = linkedin['company_state'].apply(lambda row: 'Estados Unidos' if row == '· Estados Unidos' else row)
linkedin['company_state'].value_counts()

company_state
Estados Unidos                             33
· Nueva York, NY                           11
· Seattle, WA                               8
· Washington, DC                            7
· Cincinnati, OH                            6
                                           ..
· Newport News, VA                          1
· Morris Plains, NJ                         1
· Washington DC-Baltimore y alrededores     1
· Chesterfield, VA                          1
· Santa Ana, CA                             1
Name: count, Length: 151, dtype: int64

In [420]:
def fclean_company_state(df, states):
    for i in states:
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: i if i in str(row) else row)
    return df

linkedin = fclean_company_state(linkedin, states)

In [421]:
def sclean_company_state(df, states):
    for key, value in states.items():
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: value if key in str(row) else row)
    return df

left_states = {
    'Virginia': 'VA',
    'Minnesota': 'MN',
    'Washington': 'WA',
    'San Francisco': 'CA',
    'Nebraska': 'NE',
    'California': 'CA',
    'Texas': 'TX',
    'Nueva York': 'NY',
    'Michigan': 'MI',
    'Luisiana': 'LA',
    'Florida': 'FL',
    'Maryland': 'MD',
    'Nevada': 'NV',
    'Oregón': 'OR',
    'Oklahoma': 'OK',
    'Georgia': 'GA',
    'Carolina del Sur': 'SC',
    'Delaware': 'DE',
    'Ohio': 'OH',
    'Arkansas': 'AR',
    'Iowa': 'IA',
    'Carolina del Norte': 'NC',
    'Indiana': 'IN',
    'Nashville': 'TN',
    'Massachusetts': 'MA',
    'Tennessee': 'TN',
    'Enid': 'OK'
}
linkedin = sclean_company_state(linkedin, left_states)

#### Cleaning 'company_state' column

In [422]:
linkedin.sample(5)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
250,3773899223,Data Analyst I,Centene Corporation,NH,131.900 $/año - 151.700 $/año,En remoto,Jornada completa,Sin experiencia,Más de 10.001 empleados · Hospitales y atenció...
290,3775821533,Machine Learning Engineer,Conexess Group,MO,198.200 $/año - 297.300 $/año,En remoto,Jornada completa,Sin experiencia,De 51 a 200 empleados · Servicios y consultorí...
21,3728125911,Research Engineer - FAIR Core Learning,Meta,WA,200.000 $/año - 240.000 $/año,Híbrido,Jornada completa,Director,Más de 10.001 empleados · Desarrollo de software
115,3765439827,Data Scientist - Marketing Analytics (212176),Medix Technology,Estados Unidos,75 $/h - 80 $/h,En remoto,Contrato por obra,Intermedio,De 201 a 500 empleados · Servicios y consultor...
228,3773397379,Senior Data Scientist,Charter Global,VA,75 $/h - 80 $/h,En remoto,Contrato por obra,Intermedio,De 1.001 a 5.000 empleados · Servicios y consu...


In [423]:
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto                                                                               117
Híbrido                                                                                 105
Presencial                                                                               69
En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.      2
Name: count, dtype: int64

In [424]:
def clean_remote_ratio(df, dict_):
    for key, val in dict_.items():
        df['remote_ratio'] = df['remote_ratio'].apply(lambda row: key if key in str(row) else str(row))
    return df

remote_dict = {
    'Presencial': 0,
    'Híbrido': 50,
    'En remoto': 100
}
linkedin = clean_remote_ratio(linkedin, remote_dict)
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     119
Híbrido       105
Presencial     69
nan             2
Name: count, dtype: int64

In [434]:
linkedin.loc[linkedin['remote_ratio'] == 'nan', 'remote_ratio'] = np.nan
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     119
Híbrido       105
Presencial     69
Name: count, dtype: int64

#### Cleaning 'experience_level' column

In [427]:
linkedin.sample(5)
linkedin['experience_level'].value_counts()

experience_level
Intermedio                 118
Sin experiencia             79
Algo de responsabilidad     49
Prácticas                   20
Director                    11
Ejecutivo                    2
Name: count, dtype: int64

#### Cleaning 'employment_type' column

In [428]:
def clean_employment_type(df, list_):
    for i in list_:
        df['employment_type'] = df['employment_type'].apply(lambda row: i if i in str(row) else str(row))
    return df

remote_dict = [
    'Jornada completa',
    'Contrato por obra',
    'Media jornada',
    'Prácticas'
]
linkedin = clean_employment_type(linkedin, remote_dict)
linkedin['employment_type'].value_counts()

employment_type
Jornada completa     185
Contrato por obra     54
Media jornada         31
Prácticas             25
Name: count, dtype: int64

#### Cleaning 'company_size' column

In [429]:
linkedin['company_size'].value_counts()

company_size
Más de 10.001 empleados · Hospitales y atención sanitaria                        22
De 51 a 200 empleados · Servicios y consultoría de TI                            22
De 5.001 a 10.000 empleados · Servicios y consultoría de TI                      15
Más de 10.001 empleados · Seguros                                                12
De 51 a 200 empleados · Dotación y selección de personal                         12
                                                                                 ..
De 51 a 200 empleados · Departamento de defensa y del espacio exterior            1
Más de 10.001 empleados · Fabricación de productos de defensa y espacio           1
Más de 10.001 empleados · Fabricación de maquinaria de automatización             1
De 51 a 200 empleados · Servicios de investigación                                1
De 1.001 a 5.000 empleados · Tecnología, información y medios de comunicación     1
Name: count, Length: 96, dtype: int64

In [430]:
linkedin['empleados'] = linkedin['company_size'].apply(lambda row: row.split('·')[0].strip())
linkedin['empleados'].value_counts()

empleados
Más de 10.001 empleados                                                     82
De 51 a 200 empleados                                                       59
De 1.001 a 5.000 empleados                                                  44
De 11 a 50 empleados                                                        32
De 5.001 a 10.000 empleados                                                 22
Entre 1 y 10 empleados                                                      21
De 201 a 500 empleados                                                      18
De 501 a 1.000 empleados                                                    13
Ve una comparación con los otros 287 solicitantes. Probar Premium gratis     1
Fabricación de productos de plástico                                         1
Ve una comparación con los otros 444 solicitantes. Probar Premium gratis     1
Servicios para el individuo y la familia                                     1
Name: count, dtype: int64

In [431]:
# Patrones que deseas buscar en la columna específica
patrones = [
    r"Ve una comparación con los otros \d+ solicitantes\. Probar Premium gratis",
    r"Fabricación de productos de plástico",
    r"Servicios para el individuo y la familia"
]

# Crear una expresión regular combinando los patrones con el operador OR (|)
regex_pattern = '|'.join(patrones)

# Filtrar el DataFrame por la columna específica
linkedin[linkedin['empleados'].str.contains(regex_pattern)]

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size,empleados
91,3762242381,Data Scientist,Probar Premium gratis,IL,140.000 $/año - 170.000 $/año,Híbrido,Jornada completa,Intermedio,Ve una comparación con los otros 287 solicitan...,Ve una comparación con los otros 287 solicitan...
190,3771514066,Quality Control Analyst,"Carolina Poly, Inc.",SC,120.000 $/año - 165.000 $/año,Híbrido,Jornada completa,Sin experiencia,Fabricación de productos de plástico,Fabricación de productos de plástico
270,3775200323,Data Analyst,Probar Premium gratis,MA,35 $/h - 40 $/h,En remoto,Media jornada,NaN,Ve una comparación con los otros 444 solicitan...,Ve una comparación con los otros 444 solicitan...
286,3775649749,Data Consultant ( Remote),Fresca Fresh Blanket Service,Estados Unidos,NaN,En remoto,Media jornada,Algo de responsabilidad,Servicios para el individuo y la familia,Servicios para el individuo y la familia


In [432]:
linkedin.loc[linkedin['empleados'].str.contains(regex_pattern), 'empleados'] = np.nan
linkedin['empleados'].value_counts()

empleados
Más de 10.001 empleados        82
De 51 a 200 empleados          59
De 1.001 a 5.000 empleados     44
De 11 a 50 empleados           32
De 5.001 a 10.000 empleados    22
Entre 1 y 10 empleados         21
De 201 a 500 empleados         18
De 501 a 1.000 empleados       13
Name: count, dtype: int64

#### Cleaning 'salary_range' column

In [436]:
linkedin['salary_range'].value_counts()

salary_range
140.000 $/año - 170.000 $/año    23
131.900 $/año - 151.700 $/año    21
30 $/hr                          18
60 $/h - 63 $/h                  13
170.000 $/año - 190.000 $/año    12
10.120 $/año - 12.170 $/año      11
110.000 $/año - 125.000 $/año    11
200.000 $/año - 240.000 $/año     8
78 $/h - 104 $/h                  8
48 $/h - 65 $/h                   8
85.000 $/año - 125.000 $/año      8
141.570 $/año - 173.030 $/año     7
15 $/h - 20 $/h                   7
120.000 $/año - 165.000 $/año     6
283.780 $/año - 331.640 $/año     6
198.200 $/año - 297.300 $/año     6
40 $/h - 43 $/h                   6
100.000 $/año - 150.000 $/año     5
144.000 $/año - 270.250 $/año     5
50 $/h - 57 $/h                   4
150.000 $/año - 200.000 $/año     4
75 $/h - 80 $/h                   4
95.000 $/año - 120.000 $/año      4
400.000 $/año - 700.000 $/año     3
80.000 $/año - 95.000 $/año       3
275.000 $/año - 425.000 $/año     3
100.000 $/año - 115.000 $/año     3
55 $/h - 60 $/h